# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [134]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [135]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [136]:
# confirm that our new database was created
databases = mongo.list_database_names()
databases

['admin', 'autosaurus', 'classDB', 'config', 'local', 'met', 'uk_food']

In [137]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [138]:
# review the collections in our new database
collections = db.list_collection_names()
collections

['establishments']

In [139]:
# review a document in the establishments collection
establishments_collection = db['establishments']
document = establishments_collection.find_one()
pprint(document)

{'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 289352,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2015-08-27T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('670de8c10f88cd59ccfa9af4'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289352',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001

In [140]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [141]:
# Create a dictionary for the new restaurant data
new_business = {
     "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
    
}

In [142]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_business)

In [143]:
# Check that the new restaurant was inserted
insert_result.inserted_id

ObjectId('671061e144ebcc65c7af484a')

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [144]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1}
result = establishments.find_one(query, projection)

pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('670de8c10f88cd59ccfa9b00')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [145]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"}, 
    {"$set": {"BusinessTypeID": 1}} 
)


In [146]:
# Confirm that the new restaurant was updated
restaurant = establishments.find_one({"BusinessName": "Penang Flavours"}) 
pprint(restaurant) 
 

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6710432944ebcc65c7af4843'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [147]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
dover_count = establishments.count_documents(query)
dover_count

0

In [148]:
# Delete all documents where LocalAuthorityName is "Dover""
query = {"LocalAuthorityName": "Dover"}
delete_result = establishments.delete_many(query)
delete_result.deleted_count

0

In [149]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
remaining_document = establishments.find_one(query)

# Print the result
if remaining_document:
    print("There are still documents with 'Dover'.")
else:
    print("No documents with 'Dover' remain.")

No documents with 'Dover' remain.


In [150]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

if remaining_document:
    pprint(remaining_document)
else:
    print("No documents remain in the collection.")

{'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 289352,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2015-08-27T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('670de8c10f88cd59ccfa9af4'),
 'geocode': {'latitude': 51.08084, 'longitude': 1.188537},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289352',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [151]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {
        "geocode": {"$exists": True},
        "geocode.latitude": {"$type": "string", "$ne": ""},
        "geocode.longitude": {"$type": "string", "$ne": ""}
    },
    {
        "$set": {
            "geocode.latitude": {"$convert": {"input": "$geocode.latitude", "to": "double", "onError": None}},
            "geocode.longitude": {"$convert": {"input": "$geocode.longitude", "to": "double", "onError": None}}
        }
    }
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [152]:
# Set non 1-5 Rating Values to Null
non_ratings = ["Pass"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [153]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {
        "RatingValue": {
            "$exists": True,
            "$type": "string",
            "$ne": "",
            "$nin": ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
        }
    },
    {
        "$set": {
            "RatingValue": {"$convert": {"input": "$RatingValue", "to": "int", "onError": None}}
        }
    }
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [167]:
# Check that the coordinates and rating value are now numbers
documents = list(establishments.find())

# Convert the list of documents to a DataFrame
establishments_df = pd.DataFrame(documents)

# Display the DataFrame's info
print(establishments_df.dtypes)


_id                            object
FHRSID                        float64
ChangesByServerID             float64
LocalAuthorityBusinessID       object
BusinessName                   object
BusinessType                   object
BusinessTypeID                 object
AddressLine1                   object
AddressLine2                   object
AddressLine3                   object
AddressLine4                   object
PostCode                       object
Phone                          object
RatingValue                   float64
RatingKey                      object
RatingDate                     object
LocalAuthorityCode             object
LocalAuthorityName             object
LocalAuthorityWebSite          object
LocalAuthorityEmailAddress     object
scores                         object
SchemeType                     object
geocode                        object
RightToReply                   object
Distance                      float64
NewRatingPending                 bool
meta        